In [ ]:
%matplotlib inline

import numpy as np
import h5py
from tqdm.notebook import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import scipy.stats

import fastpli.simulation
import fastpli.io


In [ ]:
# df = pd.read_pickle("output/vs_0.01_6_25/voxel_size_post_0.pkl")
df = pd.read_pickle("output/vs_0.01_6_25/voxel_size_post_1.pkl")

In [ ]:
print(df.columns)

In [ ]:
# "epa_trans",
# "epa_dir",
# "epa_ret",

sub = df[['voxel_size', 'radius', 'psi', 'omega', 'f0_inc', 'f1_rot', 'model', 'setup']]
# print(sub)
sub = sub.drop_duplicates()
for voxel_size, radius, psi, omega, f0_inc, f1_rot, model, setup in sub.itertuples(index=False):
    sub_ = (df.voxel_size == voxel_size) & (df.radius == radius) & (df.psi == psi) &(df.omega == omega) & (df.f0_inc == f0_inc) & (df.f1_rot == f1_rot) & (df.model == model) & (df.setup == setup)& (df.m > 0)
    df_ = df[sub_]
    df_ = df_.explode('epa_ret').explode('epa_ret')
    species = list(df_.species.unique())
    for i, s1 in enumerate(species):
        for s2 in species[i+1:]:
            s, p = scipy.stats.kruskal(df_[df_.species==s1].epa_ret.to_numpy(), df_[df_.species==s2].epa_ret.to_numpy())
            if p < 0.01:
                print(voxel_size, radius, psi, omega, f0_inc, f1_rot, model, setup, s1, s2, p)

print()
print()
print()
for voxel_size, radius, psi, omega, f0_inc, f1_rot, model, setup in sub.itertuples(index=False):
    sub_ = (df.voxel_size == voxel_size) & (df.radius == radius) & (df.psi == psi) &(df.omega == omega) & (df.f0_inc == f0_inc) & (df.f1_rot == f1_rot) & (df.model == model) & (df.setup == setup)& (df.m > 0)
    df_ = df[sub_]
    df_ = df_.explode('epa_ret').explode('epa_ret')
    species = list(df_.species.unique())
    for i, s1 in enumerate(species):
        for s2 in species[i+1:]:
            s, p = scipy.stats.kruskal(df_[df_.species==s1].epa_ret.to_numpy(), df_[df_.species==s2].epa_ret.to_numpy())
            if p > 0.05:
                print(voxel_size, radius, psi, omega, f0_inc, f1_rot, model, setup, s1, s2, p)


In [ ]:
df_kruskal = pd.DataFrame()

sub = df[['species', 'radius', 'psi', 'omega', 'f0_inc', 'f1_rot', 'model', 'setup']]
sub = sub.drop_duplicates()

voxel_sizes = list(df_.voxel_size.unique())
vs1 = min(voxel_sizes)

for species, radius, psi, omega, f0_inc, f1_rot, model, setup in tqdm(sub.itertuples(index=False), total=len(sub)):
    sub_ = (df.species == species) & (df.radius == radius) & (df.psi == psi) &(df.omega == omega) & (df.f0_inc == f0_inc) & (df.f1_rot == f1_rot) & (df.model == model) & (df.setup == setup)& (df.m > 0)
    df_ = df[sub_]
    df_ = df_.explode('data_diff').explode('data_diff')
    for vs2 in voxel_sizes[1:]:
        s, p = scipy.stats.kruskal(df_[df_.voxel_size==vs1].data_diff.to_numpy(), df_[df_.voxel_size==vs2].data_diff.to_numpy())

        df_kruskal = df_kruskal.append({"species":species,
                                        "radius":radius,
                                        "psi":psi,
                                        "omega":omega,
                                        "f0_inc":f0_inc,
                                        "f1_rot":f1_rot,
                                        "model":model,
                                        "setup":setup,
                                        "voxel_size": vs2,
                                        "p_value": p }, ignore_index=True)

In [ ]:
print(df_kruskal.voxel_size.unique())

In [ ]:
sns.boxplot(x="radius", y="p_value",
            hue="voxel_size",
            whis=[0, 100], width=.6,
            data=df_kruskal)
sns.stripplot(x="radius", y="p_value", data=df_kruskal,
              size=4, color=".3", linewidth=0)            